# `Symbolics.jl`
This is another fun package that I'd like to showcase here. Note that it's pretty young compared to the other packages mentioned in this tutorial, and therefore it's likely to improve even more with the years to come.(Most of this notebook has been copied from the `Symbolics.jl` [homepage](https://symbolics.juliasymbolics.org/dev/) and a [discourse thread](https://discourse.julialang.org/t/seven-lines-of-julia-examples-sought/50416/132))

## Feature summary:
Because Symbolics.jl is built into the Julia language and works with its dispatches, generic functions in Base Julia will work with symbolic expressions! Make matrices of symbolic expressions and multiply them: it will just work. Take the LU-factorization. Etc.

A general list of the features is:

- Symbolic arithmetic with type information and multiple dispatch
- Symbolic polynomials and trigonometric functions
- Pattern matching, simplification and substitution
- Differentiation
- Symbolic linear algebra (factorizations, inversion, determinants, eigencomputations, etc.)
- Discrete math (representations of summations, products, binomial coefficients, etc.)
- Logical and Boolean expressions
- Symbolic equation solving and conversion to arbitrary precision
- Support for non-standard algebras (non-commutative symbols and customizable rulesets)
- Special functions (list provided by SpecialFunctions.jl)
- Automatic conversion of Julia code to symbolic code
- Generation of (high performance and parallel) functions from symbolic expressions
- Fast automated sparsity detection and generation of sparse Jacobian and Hessians
- ...

## Building Symbolic Expressions

The way to define symbolic variables is via the `@variables` macro:


In [1]:
using Symbolics
@variables x y

2-element Vector{Num}:
 x
 y

After defining variables as symbolic, symbolic expressions, which we call a
`istree` object, can be generated by utilizing Julia expressions. For example:

In [2]:
z = x^2 + y

y + x^2

Here, `z` is an expression tree for “square `x` and add `y`”. To make an array
of symbolic expressions, simply make an array of symbolic expressions:

In [3]:
A = [x^2 + y 0 2x
     0       0 2y
     y^2 + x 0 0]

3×3 Matrix{Num}:
 y + x^2  0  2x
       0  0  2y
 x + y^2  0   0

## Derivatives

One common thing to compute in a symbolic system is derivatives. In
Symbolics.jl, derivatives are represented lazily via operations,
just like any other function. To build a differential operator, use
`Differential` like:


In [4]:
@variables t
D = Differential(t)

(::Differential) (generic function with 2 methods)

This is the differential operator $D = \frac{\partial}{\partial t}$. We can
compose the differential operator by `*`, e.g.
`Differential(t) * Differential(x)` or `Differential(t)^2`.
Now let's write down the derivative of some expression:


In [5]:
z = t + t^2
D(z)

Differential(t)(t + t^2)

Notice that this hasn't computed anything yet: `D` is a lazy operator
because it lets us symbolically represent “The derivative of $z$ with
respect to $t$”, which is useful for example when representing our
favorite thing in the world, differential equations. However, if we
want to expand the derivative operators, we'd use `expand_derivatives`:

In [6]:
expand_derivatives(D(z))

1 + 2t

## Simplification and Substitution

Symbolics interfaces with [SymbolicUtils.jl](https://github.com/JuliaSymbolics/SymbolicUtils.jl)
to allow for simplifying symbolic expressions. This is done simply
through the `simplify` command:


In [7]:
simplify(2x + 2y)

2(x + y)

This can be applied to arrays by using Julia's broadcast mechanism:

In [8]:
B = simplify.([t + t^2 + t + t^2  2t + 4t
               x + y + y + 2t     x^2 - x^2 + y^2])

2×2 Matrix{Num}:
   2(t + t^2)   6t
 x + 2(t + y)  y^2

We can then use `substitute` to change values of an expression around:

In [9]:
simplify.(substitute.(B, (Dict(x => y^2),)))

2×2 Matrix{Num}:
     2(t + t^2)   6t
 y^2 + 2(t + y)  y^2

and we can use this to interactively evaluate expressions without
generating and compiling Julia functions:

In [10]:
V = substitute.(B, (Dict(x => 2.0, y => 3.0, t => 4.0),))

2×2 Matrix{Num}:
 40.0  24.0
 16.0   9.0

Where we can reference the values via:

In [11]:
Symbolics.value.(V)

2×2 Matrix{Float64}:
 40.0  24.0
 16.0   9.0

## A fun example

In [16]:
@syms x y z
import LinearAlgebra: dot, (×)
const ∂ = Symbolics.derivative
struct ∇; end
∇(ψ::T) where T = [∂(ψ,x), ∂(ψ,y), ∂(ψ,z)]
dot(::Type{∇}, u) = ∂(u[1],x) + ∂(u[2],y) + ∂(u[3],z)
(×)(::Type{∇}, u) = [∂(u[3],y) - ∂(u[2],z), ∂(u[1],z) - ∂(u[3],x), ∂(u[2],x) - ∂(u[1],y)] 
Δ(ψ) = ∂(∂(ψ,x),x) + ∂(∂(ψ,y),y) + ∂(∂(ψ,z),z)
dot(u, ::Type{∇}) = v->map(vi->u[1]*∂(vi,x) + u[2]*∂(vi,y) + u[3]*∂(vi,z), v)

dot (generic function with 89 methods)

In [13]:
u = [y^2+z*y,x^2+2y,z*x]

3-element Vector{SymbolicUtils.BasicSymbolic{Number}}:
 y^2 + y*z
 x^2 + 2y
 x*z

In [18]:
dot(u,∇)(u)

3-element Vector{Num}:
  (z + 2y)*(x^2 + 2y) + x*y*z
 2(x^2 + 2y) + 2x*(y^2 + y*z)
      z*(x^2) + z*(y^2 + y*z)

In [20]:
dot(∇,u)

2 + x

In [21]:
∇×u

3-element Vector{Num}:
           0
       y - z
 2x - 2y - z

In [25]:
Δ(u) == dot(∇,(∇(u)))

true